In [3]:
import requests

In [4]:
standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"

In [5]:
data = requests.get(standings_url)

In [6]:
from bs4 import BeautifulSoup

In [7]:
soup = BeautifulSoup(data.text)
standings_table = soup.select('table.stats_table')[0]
links = standings_table.find_all('a')
links = [l.get("href") for l in links]
links = [l for l in links if '/squads/' in l]

In [9]:
team_urls = [f"https://fbref.com{l}" for l in links]

In [10]:
data = requests.get(team_urls[0])

In [11]:
import pandas as pd
matches = pd.read_html(data.text, match="Scores & Fixtures")[0]

In [12]:
soup = BeautifulSoup(data.text)
links = soup.find_all('a')
links = [l.get("href") for l in links]
links = [l for l in links if l and 'all_comps/shooting/' in l]

In [13]:
data = requests.get(f"https://fbref.com{links[0]}")

In [14]:
shooting = pd.read_html(data.text, match="Shooting")[0]

In [ ]:
shooting

In [15]:
shooting.columns = shooting.columns.droplevel()

In [22]:
team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date")

In [23]:
team_data

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,...,Referee,Match Report,Notes,Gls,Sh,SoT,Dist,FK,PK,PKatt
0,2020-09-21,20:15,Premier League,Matchweek 2,Mon,Away,W,3,1,Wolves,...,Andre Marriner,Match Report,NaN,3,13,8,20.3,2.0,1,1
1,2020-09-24,19:45,EFL Cup,Third round,Thu,Home,W,2,1,Bournemouth,...,Jonathan Moss,Match Report,NaN,2,19,4,NaN,NaN,0,0
2,2020-09-27,16:30,Premier League,Matchweek 3,Sun,Home,L,2,5,Leicester City,...,Michael Oliver,Match Report,NaN,2,16,5,18.8,1.0,0,0
3,2020-09-30,19:00,EFL Cup,Fourth round,Wed,Away,W,3,0,Burnley,...,Andy Madley,Match Report,NaN,3,15,8,NaN,NaN,0,0
4,2020-10-03,17:30,Premier League,Matchweek 4,Sat,Away,D,1,1,Leeds United,...,Mike Dean,Match Report,NaN,1,23,1,17.6,1.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56,2021-05-08,17:30,Premier League,Matchweek 35,Sat,Home,L,1,2,Chelsea,...,Anthony Taylor,Match Report,NaN,1,15,3,19.0,0.0,0,1
57,2021-05-14,20:00,Premier League,Matchweek 36,Fri,Away,W,4,3,Newcastle Utd,...,Kevin Friend,Match Report,NaN,4,15,6,19.0,1.0,0,0
58,2021-05-18,19:00,Premier League,Matchweek 37,Tue,Away,L,2,3,Brighton,...,Stuart Attwell,Match Report,NaN,2,8,4,13.2,1.0,0,0
59,2021-05-23,16:00,Premier League,Matchweek 38,Sun,Home,W,5,0,Everton,...,Michael Oliver,Match Report,NaN,5,21,11,15.3,0.0,0,0


In [51]:
years = list(range(2022, 2012, -1))
all_matches = []

In [52]:
standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"

In [53]:
for year in years[:1]:
    data = requests.get(standings_url)
    soup = BeautifulSoup(data.text)
    data = requests.get(standings_url)
    standings_table = soup.select('table.stats_table')[0]

    links = [l.get("href") for l in standings_table.find_all('a')]
    links = [l for l in links if '/squads/' in l]
    team_urls = [f"https://fbref.com{l}" for l in links]
    
    previous_season = soup.select("a.prev")[0].get("href")
    standings_url = f"https://fbref.com/{previous_season}"
    
    for team_url in team_urls:
        team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ")
        data = requests.get(team_url)
        matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
        soup = BeautifulSoup(data.text)
        links = [l.get("href") for l in soup.find_all('a')]
        links = [l for l in links if l and 'all_comps/shooting/' in l]
        data = requests.get(f"https://fbref.com{links[0]}")
        shooting = pd.read_html(data.text, match="Shooting")[0]
        shooting.columns = shooting.columns.droplevel()
        
        team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date")
        team_data = team_data[team_data["Comp"] == "Premier League"]
        
        team_data["Season"] = year
        team_data["Team"] = team_name
        all_matches.append(team_data)

In [54]:
len(all_matches)

20

In [55]:
match_df = pd.concat(all_matches)

In [56]:
match_df.columns = [c.lower() for c in match_df.columns]

In [63]:
match_df

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,match report,notes,sh,sot,dist,fk,pk,pkatt,season,team
1,2021-08-15,16:30,Premier League,Matchweek 1,Sun,Away,L,0,1,Tottenham,...,Match Report,NaN,18.0,4.0,16.9,1.0,0.0,0.0,2022,Manchester City
2,2021-08-21,15:00,Premier League,Matchweek 2,Sat,Home,W,5,0,Norwich City,...,Match Report,NaN,16.0,4.0,17.3,1.0,0.0,0.0,2022,Manchester City
3,2021-08-28,12:30,Premier League,Matchweek 3,Sat,Home,W,5,0,Arsenal,...,Match Report,NaN,25.0,10.0,14.3,0.0,0.0,0.0,2022,Manchester City
4,2021-09-11,15:00,Premier League,Matchweek 4,Sat,Away,W,1,0,Leicester City,...,Match Report,NaN,25.0,8.0,14.0,0.0,0.0,0.0,2022,Manchester City
6,2021-09-18,15:00,Premier League,Matchweek 5,Sat,Home,D,0,0,Southampton,...,Match Report,NaN,16.0,1.0,15.7,1.0,0.0,0.0,2022,Manchester City
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33,2022-03-13,14:00,Premier League,Matchweek 29,Sun,Away,L,1.0,2.0,Leeds United,...,Match Report,NaN,12.0,4.0,18.0,1.0,0.0,0.0,2022,Norwich City
34,2022-04-02,15:00,Premier League,Matchweek 31,Sat,Away,D,0.0,0.0,Brighton,...,Match Report,NaN,6.0,0.0,18.2,0.0,0.0,0.0,2022,Norwich City
35,2022-04-10,14:00,Premier League,Matchweek 32,Sun,Home,W,2.0,0.0,Burnley,...,Match Report,NaN,17.0,6.0,19.1,0.0,0.0,0.0,2022,Norwich City
36,2022-04-16,15:00,Premier League,Matchweek 33,Sat,Away,L,2.0,3.0,Manchester Utd,...,Match Report,NaN,15.0,4.0,19.8,1.0,0.0,0.0,2022,Norwich City
